In [2]:
import dopo
from dopo import*

In [3]:
import brightway2 as bw
import bw2data as bd

In [4]:
bd.projects.set_current("premise-validation-try1")
bw.bw2setup()

bio3=bw.Database('biosphere3')
ei39=bw.Database('ecoinvent 3.9.1 cutoff')
ei39SSP2=bw.Database('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27')

Biosphere database already present!!! No setup is needed


In [5]:
files_dict={}
files_dict['Cement']={'yaml': r'C:\Users\fried\premise_validation\dev\yamls\cement_small.yaml', 
                      'yaml identifier': 'Cement'} #yaml identifier is the name of the filter in the yaml file, in the first line.
files_dict['Steel']= {'yaml':r'C:\Users\fried\premise_validation\dev\yamls\steel_small.yaml',
                           'yaml identifier': 'Steel'}
#files_dict['Fuel']={'yaml': r'C:\Users\fried\premise_validation\dopo\yamls\fuels_small.yaml',
                   # 'yaml identifier': 'Fuels' }
files_dict

{'Cement': {'yaml': 'C:\\Users\\fried\\premise_validation\\dev\\yamls\\cement_small.yaml',
  'yaml identifier': 'Cement'},
 'Steel': {'yaml': 'C:\\Users\\fried\\premise_validation\\dev\\yamls\\steel_small.yaml',
  'yaml identifier': 'Steel'}}

In [6]:
#for comparison
premise_dict = dopo.sector_filter.process_yaml_files(files_dict, ei39SSP2)
ecoinvent_dict = dopo.sector_filter.process_yaml_files(files_dict, ei39)


Processing Cement with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Cement:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'fcb666edf2a01467e555eeff5b4a5bbb')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '36a53c174f34e672bc15b7e55563685e')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'a3c2064d83411f7963af550c04c869a1')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '86841f8c7ee2668f244d3b8e34f41932')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '3c16b45db40210cd97de6574b2f47aaf')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'df49e8f525497f2fbd56bcdc80ff0cde')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', 'f8b84f45f50d3bd7ff4feaabdb493f6a')
Processing Steel with database ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27
Activities for Steel:
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '18b0dcf01dd401e1549b3796e3786213')
  ('ei_cutoff_3.9_image_SSP2-RCP19_2050 2024-06-27', '1dffacc9e0ca08fb55c6b780d7e677dc

In [7]:
finder=dopo.methods.MethodFinder()

finder.find_and_create_method(criteria=['IPCC', '2013', 'GWP100'], exclude=['no LT'])
finder.find_and_create_method(criteria=['EN15804','Cumulative', 'non-renewable' ])
finder.find_and_create_method(criteria=['land occupation','selected'])
finder.find_and_create_method(criteria=['EN15804','fresh water'])
method_dict=finder.get_all_methods()
method_dict

{'method_1': {'object': Brightway2 Method: IPCC 2013: climate change: global warming potential (GWP100),
  'method name': ('IPCC 2013',
   'climate change',
   'global warming potential (GWP100)'),
  'short name': 'global warming potential (GWP100)',
  'unit': 'kg CO2-Eq'},
 'method_2': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: Cumulative Energy Demand - non-renewable energy resources,
  'method name': ('EN15804',
   'inventory indicators ISO21930',
   'Cumulative Energy Demand - non-renewable energy resources'),
  'short name': 'Cumulative Energy Demand - non-renewable energy resources',
  'unit': 'megajoule'},
 'method_3': {'object': Brightway2 Method: selected LCI results: resource: land occupation,
  'method name': ('selected LCI results', 'resource', 'land occupation'),
  'short name': 'land occupation',
  'unit': 'square meter-year'},
 'method_4': {'object': Brightway2 Method: EN15804: inventory indicators ISO21930: use of net fresh water,
  'method na

In [9]:
scores_premis = dopo._sector_lca_scores(premise_dict, method_dict, cutoff=0.2)

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '
Omitting activity name common prefix: 'steel production, electric, '


In [15]:
from dopo.plots_sector_lca_scores import _categorize_sheets_by_sector
from openpyxl import load_workbook
from openpyxl.chart import ScatterChart, BarChart, Reference, Series
def stacked_bars_xcl(filepath_workbook, column_positions, current_row_dot_plot):
    """
    Creates stacked bar charts for each sector in an Excel workbook.

    This function reads an Excel workbook, categorizes sheets by sector, and creates 
    stacked bar charts to visualize data contributions. The generated charts are added 
    to a new or existing sheet within the workbook.

    Parameters
    ----------
    filepath_workbook : str
        Path to the Excel workbook file.
    index_positions : dict
        A dictionary containing column index positions for the data required to create 
        the charts for each worksheet.
    current_row_dot_plot : int
        The row number in the chart sheet where the dot plots ended, used to determine 
        the starting row for the stacked bar charts.

    Returns
    -------
    int
        The row position where the last chart was placed.
    """
    # Categorize sheets by sector
    worksheet_dict = _categorize_sheets_by_sector(filepath_workbook)
    
    # Load the workbook
    wb = load_workbook(filepath_workbook)
    
    # Iterate over each sector and its associated worksheets
    for sector, worksheet_names in worksheet_dict.items():
        
        # Create or get the chart sheet for the current sector
        chart_sheet_name = f"{sector}_charts"
        if chart_sheet_name in wb.sheetnames:
            ws_charts = wb[chart_sheet_name]
        else:
            ws_charts = wb.create_sheet(chart_sheet_name)
                
        # Initial position for the first chart
        chart_height = 30  # Number of rows a chart occupies
        chart_width = 12   # Number of columns a chart occupies
        current_row = current_row_dot_plot + chart_height  # Start placing charts from row after dot plots
        current_col = 1  # Start placing charts from column 1
        charts_per_row = 3  # Number of charts per row
        
        # Iterate over each worksheet name in the current sector
        for i, worksheet_name in enumerate(worksheet_names):
            ws = wb[worksheet_name]

            # Find the key in index_positions that contains worksheet_name
            matching_key = None
            for key in column_positions.keys():
                if worksheet_name in key:
                    matching_key = key
                    break

            if not matching_key:
                print(f"Warning: No matching key found for worksheet '{worksheet_name}'. Skipping...")
                continue

            # Retrieve the column positions from the index_positions dictionary
            positions = column_positions[matching_key]

            # Find min_row, max_row, and max_column
            max_row = ws.max_row
            max_column = ws.max_column
            input_min_col = positions.get("first_input", None) + 1
            rank_col = positions.get("rank", None) + 1
            method_col = positions.get("method", None) + 1
            method_unit_col = positions.get("method unit", None) + 1

            # Create a BarChart object for the stacked bar chart
            chart = BarChart()
            chart.type = "bar"
            chart.style = 2
            chart.grouping = "stacked"
            chart.overlap = 100

            # Chart titles
            method_value = ws.cell(row=2, column=method_col).value
            chart.title = f"Inputs contributions to {method_value} LCA score for sector {sector}"

            method_unit_value = ws.cell(row=2, column=method_unit_col).value
            chart.y_axis.title = f"{method_unit_value}"
            chart.x_axis.title = 'activity rank'

            # Avoid overlap
            chart.title.overlay = False
            chart.x_axis.title.overlay = False
            chart.y_axis.title.overlay = False 
            chart.legend.overlay = False

            # Define data for the stacked bar chart
            data = Reference(ws, min_col=input_min_col, min_row=1, max_row=max_row, 
                             max_col=max_column)
            cats = Reference(ws, min_col=rank_col, min_row=2, max_row=max_row)

            chart.add_data(data, titles_from_data=True)
            chart.set_categories(cats)
            chart.shape = 4

            # Modify each series in the chart to disable the inversion of negative values 
            for series in chart.series:
                series.invertIfNegative = False

            # y-axis ticks
            chart.y_axis.tickLblPos = "nextTo"
            chart.y_axis.delete = False  # Ensure axis is not deleted
            chart.y_axis.number_format = '0.000'

            # Adjust X-axis properties
            chart.x_axis.tickLblPos = "low" # "nextTo"
            chart.x_axis.majorGridlines = None 
            chart.x_axis.tickMarkSkip = 1  # Show all tick marks
            chart.x_axis.tickLblSkip = 1  # Show all labels

            chart.x_axis.scaling.orientation = "minMax"
            chart.x_axis.crosses = "autoZero"
            chart.x_axis.axPos = "b"
            chart.x_axis.delete = False

            # Adjust chart dimensions
            chart.width = 20  # Width of the chart
            chart.height = 14  # Height of the chart

            # Add the chart to the chart worksheet
            # Calculate the position for this chart
            position = ws_charts.cell(row=current_row, column=current_col).coordinate
            ws_charts.add_chart(chart, position)
            
            # Update position for the next chart
            current_col += chart_width + 1
            if (i + 1) % charts_per_row == 0:  # Move to the next row after placing `charts_per_row` charts
                current_row += chart_height + 1
                current_col = 1  # Reset to the first column

        # Move the chart sheet to the first position
        wb._sheets.remove(ws_charts)
        wb._sheets.insert(0, ws_charts)
        
    # Save the workbook with the charts added
    wb.save(filepath_workbook)
    
    return current_row


In [16]:
#Converting the lca scores and input tables to excel worksheet adding statistics and more. Storing column positions (index) in variable.
column_positions=_sector_lca_scores_to_excel(scores_premis, '0409_v3.xlsx')

key in method_dfs
Cement_global_warming_potential_(gwp100)
key in method_dfs
Cement_cumulative_energy_demand_-_non-renewable_energy_resources
key in method_dfs
Cement_land_occupation
key in method_dfs
Cement_use_of_net_fresh_water
key in method_dfs
Steel_global_warming_potential_(gwp100)
key in method_dfs
Steel_cumulative_energy_demand_-_non-renewable_energy_resources
key in method_dfs
Steel_land_occupation
key in method_dfs
Steel_use_of_net_fresh_water


In [17]:
#Generating dot plots and stacked bars in the excel storing the location of last plot in variable.
last_row= dot_plots_xcl('0409_v3.xlsx', column_positions)

# #Comparing premise and ecoinvent database by calculating lca scores, storing table sin excel and plotting barcharts. Again storing column positions in variable.
# column_positions_compare=relative_changes_db(ecoinvent_dict, premise_dict, method_dict, '0309_v8.xlsx')
# barchart_compare_db_xcl('0309_v8.xlsx', column_positions_compare, current_row_stacked_bar)

In [18]:
current_row_stacked_bar=stacked_bars_xcl('0409_v3.xlsx', column_positions, last_row)

In [9]:
sector_lca_scores_plots(premise_dict, method_dict, '0409_v2.xlsx')

Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'cement production, '
Omitting activity name common prefix: 'steel production, electric, '


KeyboardInterrupt: 

In [ ]:
database_comparison_plots(ecoinvent_dict, premise_dict, method_dict,'0409_v1.xlsx',current_row=93)

df_ei
                      activity  \
0  cement production, Portland   
1  cement production, Portland   
2  cement production, Portland   
3  cement production, Portland   
4  cement production, Portland   
5  cement production, Portland   
6  cement production, Portland   

                                        activity key reference product  \
0  (ecoinvent 3.9.1 cutoff, 3c16b45db40210cd97de6...  cement, Portland   
1  (ecoinvent 3.9.1 cutoff, 36a53c174f34e672bc15b...  cement, Portland   
2  (ecoinvent 3.9.1 cutoff, a3c2064d83411f7963af5...  cement, Portland   
3  (ecoinvent 3.9.1 cutoff, f8b84f45f50d3bd7ff4fe...  cement, Portland   
4  (ecoinvent 3.9.1 cutoff, 86841f8c7ee2668f244d3...  cement, Portland   
5  (ecoinvent 3.9.1 cutoff, df49e8f525497f2fbd56b...  cement, Portland   
6  (ecoinvent 3.9.1 cutoff, fcb666edf2a01467e555e...  cement, Portland   

  location                             method method unit     total  \
0       PE  global warming potential (GWP100)   kg CO2-Eq